<a href="https://colab.research.google.com/github/damilojohn/Playlist_Generator/blob/main/Generating_Afrobeats_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#installing necessary libraries
!pip install sentence-transformers
!pip install datasets 
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 2.9 MB/s 
     |████████████████████████████████| 5.8 MB 30.7 MB/s 
     |████████████████████████████████| 1.3 MB 39.6 MB/s 
     |████████████████████████████████| 182 kB 39.1 MB/s 
     |████████████████████████████████| 7.6 MB 44.3 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=d351f4d07c8a0bfcfd57ca77f5e4dd472e17dd29d5079d46ae6f1407ec5fa21f
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence-transformers
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 5.0 MB/s 
     |████████████████████████████████| 212 kB 35.6 MB/s 
     |████████████████████████████████| 132 kB 54.3 MB/s 
     |██████

In [2]:
#text file containing lyrics
!wget https://raw.githubusercontent.com/damilojohn/Playlist_Generator/main/model_embedding.txt

--2022-12-03 21:00:13--  https://raw.githubusercontent.com/damilojohn/Playlist_Generator/main/model_embedding.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 149618 (146K) [text/plain]
Saving to: ‘model_embedding.txt’

model_embedding.txt 100%[===================>] 146.11K  --.-KB/s    in 0.02s   

2022-12-03 21:00:13 (6.51 MB/s) - ‘model_embedding.txt’ saved [149618/149618]



In [3]:
#setting up wandb
!wandb login
import wandb

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [4]:
config = {
    'Name' : 'Embeddings for afrobeats lyrics',
    'Dataset' : 'spotify api lyrics',
    'Model' : 'msmarco-miniLM-pretrained',
    'notes':'No text cleaning was performed yet.creating a baseline performance'
}
wandb.init(project='Playlist Generator',entity='damilojohn',config=config)

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: damilojohn. Use `wandb login --relogin` to force relogin


In [5]:
from sentence_transformers import SentenceTransformer,util
import pandas as pd 
import transformers
import pickle 
from huggingface_hub import hf_hub_download
from datasets import load_dataset

In [6]:
#loading song lyrics into a dataset object 
data_path = '/content/model_embedding.txt'
songs = load_dataset('text',
                     data_files={'train':[data_path]})

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-b352a03421cf52ca/0.0.0/99cc88223027054f94ce0c7fd69d10eb172910fa0615671283a3c8e5e7af2f9c. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
lyrics = open('/content/model_embedding.txt','r')
lyrics_txt = lyrics.read()
lyrics.close()
lyrics_txts = list(lyrics_txt.split('\\n'))
print(lyrics_txts)

['["[Intro: Wizkid]', 'Oh-na-na, omo je ka shi ge ge, woah', "Oh-na-na, omo ge wa j'ogede", 'Oh-na-na, girl, I no go let you go', 'Oh-na-na, omo je ka shi ge ge, woah', '', '[Verse 1: Tems]', "Say I wanna leave you in the mornin', but I need you now, yeah, yeah", "I find you, I give you all you're needing, I know you what you like", "I feel it comin', time is of the essence", 'I tried to teach you, but I need some lessons', "I need to give it all, I tried to leave, but I can't", "I don't know why you're the one, turn me out of my mind", '', '[Chorus: Tems]', "You don't need no other body", "You don't need no other body", 'Only you fi hold my body', 'Only you fi hold my body', "You don't need no other body", '', '[Verse 2: Wizkid]', 'Say na me dey mess up your mind', 'And na me dey make you free up your mind', 'Say your body talk to me nice', 'Say na my love, you dey need for your life', 'Yeah our love no be lie', 'See yeah we dey together yeah day and night', 'Yeah if I leave you go kp

In [8]:
embedder = SentenceTransformer('msmarco-MiniLM-L-6-v3')

Downloading:   0%|          | 0.00/736 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.68k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/627 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/430 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [30]:
lyrics_txts[3],lyrics_txts[11]

('Oh-na-na, girl, I no go let you go',
 "I need to give it all, I tried to leave, but I can't")

In [9]:
prompt = lyrics_txts[3]
prompt2 = lyrics_txts[11]
first_embed = embedder.encode(prompt,convert_to_tensor=True)
second_embed = embedder.encode(prompt2,convert_to_tensor=True)
score = util.semantic_search(query_embeddings=first_embed,corpus_embeddings=second_embed)
score

[[{'corpus_id': 0, 'score': 0.3610694706439972}]]

In [10]:
def generate_embeddings(lyrics):
  embeds = [] 
  for lyric in lyrics:
    embed = embedder.encode(lyric,convert_to_tensor=True)
    embeds.append(embed)
  return embeds


In [11]:
embeds = generate_embeddings(lyrics_txts)

In [13]:
embeds[0]

tensor([-0.2114,  0.6710,  0.1919,  0.2407,  0.1931, -0.4357, -0.0235,  0.0650,
        -0.1051,  0.0433, -0.4696,  0.4317,  0.6080, -0.3825,  0.0391, -0.3282,
        -0.6902,  0.2177,  0.5765, -0.6483,  0.9303,  0.4771,  0.6173, -0.6084,
         0.2408, -0.0332,  0.2396,  0.2105, -0.0598,  0.1608,  0.0501,  0.2074,
         0.4115,  0.3523, -0.6365,  0.1564, -0.6804, -0.4075, -0.2437,  0.1547,
        -0.2494,  0.1454, -0.5928,  0.3234, -0.5376,  0.1075,  0.0215, -0.3689,
        -0.0292,  0.2111, -0.4206,  0.1119, -0.1285,  0.3759,  0.4949,  0.4693,
        -0.4311, -0.1600,  0.0446, -0.1524,  0.1754, -0.2726, -0.0863,  0.6051,
         0.6541,  0.3978, -0.0063, -0.2508,  0.2437, -0.0821, -0.2881, -0.2882,
         0.1041,  0.4002, -0.3836,  0.1001, -0.0442, -0.4031, -0.0080,  0.4719,
         0.7796, -0.0834, -0.1609,  0.0293,  0.1065, -0.1355, -0.1328, -0.2787,
        -0.2052, -0.0433, -0.4780, -0.2449, -0.3327,  0.0440, -0.3409, -0.0167,
         0.5269,  0.2891,  0.0099,  0.55